# Naive Bayes
Naive Bayes methods are a set of supervised learning algorithms based on applying Bayes’ theorem with the “naive” **assumption of conditional independence between every pair of features given the value of the class variable.** Bayes’ theorem states the following relationship, given class variable $y$ and dependent feature vector $x_1$ through $x_n$:
$$
P(y \mid x_1, \dots, x_n) = \frac{P(y) P(x_1, \dots, x_n \mid y)}{P(x_1, \dots, x_n)}
$$
_Using the naive conditional independence assumption_ that
$$
P(x_i | y, x_1, \dots, x_{i-1}, x_{i+1}, \dots, x_n) = P(x_i | y),
$$

for all $i$, this relationship is simplified to
$$
P(y \mid x_1, \dots, x_n) = \frac{P(y) \prod_{i=1}^{n} P(x_i \mid y)}
                                 {P(x_1, \dots, x_n)}
$$

Since $P(x_1,\dots,x_n)$ is constant given the input, we can use the following classification rule:
$$
\begin{align}\begin{aligned}P(y \mid x_1, \dots, x_n) \propto P(y) \prod_{i=1}^{n} P(x_i \mid y)\\\Downarrow\\\hat{y} = \arg\max_y P(y) \prod_{i=1}^{n} P(x_i \mid y),\end{aligned}\end{align}
$$

and we can use Maximum A Posteriori (MAP) estimation to estimate $P(y)$ and $P(x_i | y)$; the former is then the relative frequency of class $y$ in the trainin set.

The different naive Bayes classifiers differ mainly by the assumptions they make regarding the distribution of $P(x_i|y)$.

In spite of their apparently over-simplified assumptions, naive Bayes classifiers have worked quite well in many real-world situations, famously document classification and spam filtering. They require a small amount of training data to estimate the necessary parameters.

Naive Bayes learners and classifiers can be extremely fast compared to more sophisticated methods. The decoupling of the class conditional feature distributions means that each distribution can be independently estimated as a one dimensional distribution. This in turn helps to alleviate problems stemming from the curse of dimensionality.

On the flip side, although naive Bayes is known as a decent classifier, it is known to be a bad estimator, so the probability outputs from `predict_proba` are not to be taken too seriously.

## Gaussian Naive Bayes

GaussianNB implements the Gaussian Naive Bayes algorithm for classification. The likelihood of the features is assumed to be Gaussian:
$$
P(x_i \mid y) = \frac{1}{\sqrt{2\pi\sigma^2_y}} \exp\left(-\frac{(x_i - \mu_y)^2}{2\sigma^2_y}\right)
$$
The parameters $\sigma_y$ and $\mu_y$ are estimated using maximum likelihood. 

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points: %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))


Number of mislabeled points out of a total 75 points: 4


## Multinomial Naive Bayes
MultinomialNB implements the naive Bayes algorithm for multinomially distributed data, and is one of the two classic naive Bayes variants used in text classification (where the data are typically represented as word vector counts, although tf-idf vectors are also known to work well in practice). The distribution is parametrized by vectors $\theta_y = (\theta_{y1},\ldots,\theta_{yn})$ for each class $y$, where $n$ is the number of features (in text classification, the size of the vocabulary) and $\theta_{yi}$ is the probability $P(x_i|y)$ of feature $i$ appearing in a sample belonging to class $y$

The parameter $\theta_i$ is estimated by a smoothed version of maximum likelihood, i.e. relative frequency counting:
$$
\hat{\theta}_{yi} = \frac{ N_{yi} + \alpha}{N_y + \alpha n}
$$
where $N_{yi} = \sum_{x \in T} x_i$ is the number of times feature $i$ appears in a sample of class $y$ in the training set $T$, and $N_{y} = \sum_{i=1}^{n} N_{yi}$ is the total count of all features for class $y$

The smoothing priors $\alpha \geq 0$ accounts for features not present in the learning samples and prevents zero probabilities in further computations. Setting $\alpha=1$ is called Laplace smoothing, while $\alpha < 1$ is called Lidstone smoothing.

## Complement Naive Bayes (CNB)
ComplementNB implements the complement naive Bayes (CNB) algorithm. **CNB is an adaptation of the standard multinomial naive Bayes (MNB) algorithm that is particularly suited for imbalanced data sets.** Specifically, CNB uses statistics from the complement of each class to compute the model’s weights. The inventors of CNB show empirically that the parameter estimates for CNB are more stable than those for MNB. Further, _CNB regularly outperforms MNB_ (often by a considerable margin) on text classification tasks. The procedure for calculating the weights is as follows:
$$
\begin{align}\begin{aligned}\hat{\theta}_{ci} = \frac{\alpha_i + \sum_{j:y_j \neq c} d_{ij}}
{\alpha + \sum_{j:y_j \neq c} \sum_{k} d_{kj}}\\w_{ci} = \log \hat{\theta}_{ci}\\w_{ci} = \frac{w_{ci}}{\sum_{j} |w_{cj}|}\end{aligned}\end{align}
$$

where the summations are over all documents $j$ not in class $c$, $d_{ij}$ is either the count or tf-idf value of term $i$ in document $j$, $\alpha_i$ is a smoothing hyperparameter like that found in MNB, and $\alpha = \sum_i\alpha_i$. The second normalization addresses the tendency for longer documents to dominate parameter estimates in MNB. The classification rule is:
$$
\hat{c} = \arg\min_c \sum_{i} t_i w_{ci}
$$

i.e., a document is assigned to the class that is the _poorest_ complement match.

## Categorical Naive Bayes
CategoricalNB implements the categorical naive Bayes algorithm **for categorically distributed data.** It assumes that each feature, which is described by the index $i$, has its own categorical distribution.

For each feature $i$ in the training set $X$, CategoricalNB estimates a categorical distribution for each feature i of X conditioned on the class y. The index set of the samples is defined as $J = \{ 1, \dots, m \}$, with $m$ as the number of samples.

The probability of category $t$ in feature $i$ given class $c$ is estimated as:
$$
P(x_i = t \mid y = c \: ;\, \alpha) = \frac{ N_{ti|c} + \alpha}{N_{c} +
                                       \alpha n_i},
$$
where $N_{ti|c} = |\{j \in J \mid x_{ij} = t, y_j = c\}|$ (cardinalità del set di tutti i samples tali per cui la feature $i$ nel sample $j$ è $t$ e la classe del campione $j$ è $c$) is the number of times category $t$ appears in the samples $x_i$, which belong to class $c$, $N_{c} = |\{ j \in J\mid y_j = c\}|$ is the number of samples with class c, $\alpha$ is a smoothing parameter and $n_i$ is the number of available categories of feature $i$. 

CategoricalNB assumes that the sample matrix $X$ is encoded (for instance with the help of `OrdinalEncoder`) such that all categories for each feature $i$ are represented with numbers $0,\dots,n_i-1$ where $n_i$ is the number of available categories of feature $i$.

In [16]:
import numpy as np
rng = np.random.RandomState(0)
X = rng.randint(5, size=(6,100)) #5 is the maximum value (exluded). 
# If I type 1,5 the values will be between 1 included and 5 excluded
y = np.array(list(range(1,7)))

In [21]:
X[2] #== X[2:3]

array([1, 3, 4, 3, 3, 1, 1, 1, 3, 2, 2, 4, 3, 1, 2, 3, 1, 4, 2, 2, 0, 1,
       0, 4, 0, 3, 2, 0, 0, 1, 1, 0, 3, 0, 3, 0, 4, 4, 0, 1, 3, 4, 3, 2,
       1, 1, 2, 1, 4, 2, 4, 4, 2, 3, 3, 1, 2, 4, 3, 3, 1, 1, 1, 1, 2, 0,
       3, 1, 4, 3, 1, 1, 2, 0, 0, 2, 4, 0, 4, 3, 3, 0, 0, 3, 3, 0, 3, 0,
       2, 1, 0, 4, 0, 3, 4, 4, 1, 0, 2, 4])

In [15]:
from sklearn.naive_bayes import CategoricalNB
clf = CategoricalNB().fit(X, y)
print(clf.predict(X[2:3]))

[3]


In [11]:
X

array([[1, 4, 2, 1, 4, 4, 4, 4, 2, 4, 2, 3, 1, 4, 3, 1, 1, 1, 3, 2, 3, 4,
        4, 3, 1, 2, 2, 2, 2, 1, 2, 1, 4, 1, 4, 2, 3, 4, 4, 1, 3, 4, 1, 2,
        4, 2, 4, 4, 3, 4, 1, 2, 2, 2, 4, 1, 4, 3, 1, 4, 4, 3, 4, 3, 4, 1,
        3, 1, 1, 1, 2, 2, 3, 1, 1, 2, 4, 1, 2, 3, 3, 4, 1, 2, 2, 4, 2, 2,
        4, 3, 4, 4, 3, 3, 4, 1, 3, 4, 2, 1],
       [2, 3, 1, 4, 1, 3, 1, 4, 4, 1, 4, 1, 1, 1, 1, 3, 4, 1, 4, 3, 4, 4,
        2, 2, 2, 1, 2, 2, 2, 4, 1, 4, 2, 3, 1, 2, 3, 1, 3, 1, 2, 4, 3, 3,
        2, 1, 4, 2, 2, 4, 1, 3, 3, 4, 3, 4, 4, 4, 3, 2, 3, 3, 4, 3, 4, 4,
        3, 3, 4, 1, 2, 3, 4, 3, 2, 3, 2, 1, 3, 3, 4, 1, 4, 3, 4, 1, 1, 3,
        1, 3, 4, 3, 3, 4, 1, 1, 1, 2, 3, 1],
       [2, 3, 4, 3, 3, 4, 2, 1, 1, 1, 4, 4, 1, 3, 3, 1, 1, 4, 1, 2, 3, 1,
        1, 4, 2, 4, 1, 3, 2, 2, 2, 4, 1, 1, 1, 2, 2, 3, 2, 1, 1, 2, 3, 2,
        2, 4, 2, 1, 1, 1, 4, 4, 2, 3, 2, 4, 2, 4, 1, 1, 4, 2, 3, 1, 3, 4,
        4, 3, 4, 2, 2, 2, 4, 3, 1, 1, 2, 1, 4, 3, 4, 4, 3, 1, 3, 4, 3, 3,
        2, 2, 4, 4, 1,

## Out-of-core naive Bayes model fitting
Naive Bayes models can be used to tackle large scale classification problems for which the full training set might not fit in memory. To handle this case, MultinomialNB, BernoulliNB, and GaussianNB expose a `partial_fit` method that can be used incrementally as done with other classifiers as demonstrated in [Out-of-core classification of text documents](https://scikit-learn.org/stable/auto_examples/applications/plot_out_of_core_classification.html#sphx-glr-auto-examples-applications-plot-out-of-core-classification-py). All naive Bayes classifiers support sample weighting.

Contrary to the `fit` method, the first call to `partial_fit` needs to be passed the list of all the expected class labels.

For an overview of available strategies in scikit-learn, see also the [out-of-core learning documentation](https://scikit-learn.org/stable/computing/scaling_strategies.html#scaling-strategies).

**Note:** The `partial_fit` method call of naive Bayes models introduces some computational overhead. It is recommended to use data chunk sizes that are as large as possible, that is as the available RAM allows.